In [2]:
!pip install bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl si

In [3]:

from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)

In [5]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [6]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

In [7]:
model_name='microsoft/phi-2'
device_map = {"": 0}
original_model = AutoModelForCausalLM.from_pretrained(model_name,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

In [43]:
import pandas as pd
from datasets import load_dataset, DatasetDict,Dataset

# Load the SNLI dataset
ds = load_dataset("stanfordnlp/snli")

# Create training set by selecting every 550th sample from 550k total
train_samples = Dataset.from_dict(ds["train"].select(range(0, len(ds["train"]), 550))[:1000])

# Create testing set by selecting every 100th sample from 10k total in test set
test_samples = Dataset.from_dict(ds["test"].select(range(0, len(ds["test"]), 100))[:100])

# Create validation set by selecting every 100th sample from 10k total in validation set
validation_samples = Dataset.from_dict(ds["validation"].select(range(0, len(ds["validation"]), 100))[:100])

# Combine them into a DatasetDict
custom_dataset = DatasetDict({
    "train": train_samples,
    "test": test_samples,
    "validation": validation_samples
})

# Confirm dataset sizes
print(f"Training samples: {len(custom_dataset['train'])}")
print(f"Testing samples: {len(custom_dataset['test'])}")
print(f"Validation samples: {len(custom_dataset['validation'])}")

Training samples: 1000
Testing samples: 100
Validation samples: 100


In [10]:
# %%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 100

hypothesis = ds['test'][index]['hypothesis']
premise = ds['test'][index]['premise']
label = ds['test'][index]['label']

formatted_prompt = f"""Premise: "{premise}"
Hypothesis: "{hypothesis}"

Is the relationship between the premise and hypothesis:
- 0 for entailment
- 1 for neutral
- 2 for contradiction

Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis:
Output:
"""

print(formatted_prompt)

Premise: "A woman within an orchestra is playing a violin."
Hypothesis: "A woman is playing the violin."

Is the relationship between the premise and hypothesis:
- 0 for entailment
- 1 for neutral
- 2 for contradiction

Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis:
Output: 



In [14]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

def gen(model,p, maxlen=100, sample=True):
    toks = eval_tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

In [18]:
# formatted_prompt = f"Instruct: Summarize the following conversation.\n{prompt}\nOutput:\n"
res = gen(original_model,formatted_prompt,100,)
#print(res[0])
output = res[0].split('Output:')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE answer:\n{label}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Premise: "A woman within an orchestra is playing a violin."
Hypothesis: "A woman is playing the violin."

Is the relationship between the premise and hypothesis:
- 0 for entailment
- 1 for neutral
- 2 for contradiction

Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis:
Output: 

---------------------------------------------------------------------------------------------------
BASELINE answer:
0

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
 
1



In [50]:
def create_prompt_formats(sample):
    prompt = f"""Premise: "{sample['premise']}"
  Hypothesis: "{sample['hypothesis']}"
  Is the relationship between the premise and hypothesis:
    - 0 for entailment
    - 1 for neutral
    - 2 for contradiction
      """

    INTRO_BLURB = "Below is an instruction that describes a task. Provide a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Read the Premise and the hypothesis. Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis:"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"

    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    response = f"{RESPONSE_KEY}\n{sample['label']}"
    end = f"{END_KEY}"

    parts = [part for part in [blurb, instruction,prompt, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    sample["text"] = formatted_prompt

    return sample

In [51]:
print(create_prompt_formats(custom_dataset['train'][0])['text'])


Below is an instruction that describes a task. Provide a response that appropriately completes the request.

### Instruct: Read the Premise and the hypothesis. Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis:

Premise: "A person on a horse jumps over a broken down airplane."
  Hypothesis: "A person is training his horse for a competition."
  Is the relationship between the premise and hypothesis:
    - 0 for entailment
    - 1 for neutral
    - 2 for contradiction
      

### Output:
1

### End


In [57]:
from functools import partial

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

# SOURCE https://github.com/databrickslabs/dolly/blob/master/training/trainer.py
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)#, batched=True)

    # Apply preprocessing to each batch of the dataset & and remove 'instruction', 'context', 'response', 'category' fields
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched=True,
        remove_columns=['premise', 'hypothesis', 'label'],
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

{'premise': 'A person on a horse jumps over a broken down airplane.',
 'hypothesis': 'A person is training his horse for a competition.',
 'label': 1}

In [58]:
max_length = get_max_length(original_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length,seed, custom_dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length,seed, custom_dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [60]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(train_dataset)

Shapes of the datasets:
Training: (1000, 3)
Validation: (100, 3)
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 1000
})


In [61]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [62]:
print(original_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiSdpaAttention(
          (q_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (dense): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_la

In [63]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model)

peft_model = get_peft_model(original_model, config)

In [64]:
print(print_number_of_trainable_model_parameters(peft_model))


trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


In [ ]:
print(peft_model)

In [68]:
output_dir = './peft-dialogue-summary-training/final-checkpoint'
import transformers

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=1250,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    num_train_epochs=5
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [69]:
peft_trainer.train()

Step,Training Loss,Validation Loss
25,1.407100,0.547509
50,0.405700,0.511140
75,0.579200,0.491199
100,0.390900,0.498927
125,0.566400,0.486833
150,0.392500,0.491474
175,0.534500,0.489159
200,0.355000,0.489285
225,0.541300,0.480775
250,0.371500,0.487464


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=1250, training_loss=0.4512526264190674, metrics={'train_runtime': 2869.6732, 'train_samples_per_second': 1.742, 'train_steps_per_second': 0.436, 'total_flos': 1.08750016461312e+16, 'train_loss': 0.4512526264190674, 'epoch': 5.0})

In [ ]:
from google.colab import files

files.download('/content/peft-dialogue-summary-training')

In [71]:
del original_model
del peft_trainer
torch.cuda.empty_cache()

In [72]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

base_model_id = "microsoft/phi-2"
base_model = AutoModelForCausalLM.from_pretrained(base_model_id,
                                                      device_map='auto',
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [73]:
eval_tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [74]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "/content/peft-dialogue-summary-training/final-checkpoint/checkpoint-1000",torch_dtype=torch.float16,is_trainable=False)

In [109]:
%%time
from transformers import set_seed
seed = 42
set_seed(seed)

index = 500

hypothesis = ds['test'][index]['hypothesis']
premise = ds['test'][index]['premise']
label = ds['test'][index]['label']

formatted_prompt = f"""
Premise: "{premise}"
Hypothesis: "{hypothesis}"
Is the relationship between the premise and hypothesis:
  - 0 for entailment
  - 1 for neutral
  - 2 for contradiction
Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis
Output:
      """

print(formatted_prompt)


Premise: "many children play in the water."
Hypothesis: "The children are playing mini golf."
Is the relationship between the premise and hypothesis:
  - 0 for entailment
  - 1 for neutral
  - 2 for contradiction
Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis
Output:
      
CPU times: user 3.6 ms, sys: 51 µs, total: 3.65 ms
Wall time: 4.33 ms


In [110]:
peft_model_res = gen(ft_model,formatted_prompt,100,)
peft_model_output = peft_model_res[0].split('Output:\n')[1].split('End of Output.')[0]


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE answer:\n{label}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{peft_model_output}')

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Premise: "many children play in the water."
Hypothesis: "The children are playing mini golf."
Is the relationship between the premise and hypothesis:
  - 0 for entailment
  - 1 for neutral
  - 2 for contradiction
Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis
Output:
      
---------------------------------------------------------------------------------------------------
BASELINE answer:
2

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
        2

End of Output.

Question 2:
Premise: "A man is standing in front of a large building."
    Hypothesis: "The man is standing outside."
    Is the relationship between the premise and hypothesis:
      - 0 for entailment
      - 1 for neutral
      - 2 for contradiction
        

Answer:
1

E

In [111]:
peft_model_output = peft_model_res[0].split('Output:\n')[1].split('End of Output.')[0]


dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'BASELINE answer:\n{label}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{peft_model_output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Premise: "many children play in the water."
Hypothesis: "The children are playing mini golf."
Is the relationship between the premise and hypothesis:
  - 0 for entailment
  - 1 for neutral
  - 2 for contradiction
Answer with the correct number (0, 1, or 2) based on the relationship between the premise and hypothesis
Output:
      
---------------------------------------------------------------------------------------------------
BASELINE answer:
2

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
        2


